In [62]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import date

warnings.filterwarnings("ignore")

**1. Get an understanding of the data**

In [63]:
# Load in data
train, test = pd.read_csv("train.csv"), pd.read_csv("test.csv")

# Stack for consistent preprocessing
all = pd.concat([train, test], axis=0)

# Preview
print(f"{len(train)} training rows, {len(test)} test rows, {train.shape[1]} out-of-the-box features")
all.info() # or all.head(), all.describe()

1460 training rows, 1459 test rows, 81 out-of-the-box features
<class 'pandas.core.frame.DataFrame'>
Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 

**2. Deal with missing data; pre-processing**

In [ ]:
# To make things easier, you can just drop rows with missing categorical features (so long as there aren't too many)

# Get names of columns with any amount of missing data
problematic_columns = all.columns[all.isnull().any()]
print(problematic_columns)

# Note which column(s) need special impute logic, if applicable
# i.e. we can use YearBlt as a close approximation to GarageYrBlt

# Separate the REST of the columns that need imputation into (a) categorical and (b) quantitative ones --> exclude SalePrice bc that's what we're predicting
categorical_missing_cols = ["MSZoning", "Alley", "Utilities", "Exterior1st", "Exterior2nd", "MasVnrType", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Electrical", 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']

quantitative_missing_cols_to_categories = {
    "LotFrontage": "LotConfig",
    "MasVnrArea": "MasVnrType",
    "BsmtFinSF1": "BsmtFinType1", 
    "BsmtFinSF2": "BsmtFinType2", 
    "BsmtUnfSF": "BsmtQual", 
    "TotalBsmtSF": "BsmtQual", 
    "BsmtFullBath": "BsmtQual", 
    "BsmtHalfBath": "BsmtQual", 
    'GarageCars': "GarageType",
    'GarageArea': "GarageType",
}

Index(['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SalePrice'],
      dtype='object')


In [65]:
# If possible, explicitly convert "numerical" categorical columns to "category" dtype --> takes more attention to detail
all["MSSubClass"] = all["MSSubClass"].astype("category")
all["MoSold"] = all["MoSold"].astype("category")

In [66]:
# CATEGORICAL IMPUTATION

# Always do special logic first
# For categorical columns with enormous amounts of missing data, instead of imputing with the (first) mode, create a new category
most_missing = all[categorical_missing_cols].isnull().sum()
new_category_needed = most_missing[most_missing > 0.7 * len(all)]
print(new_category_needed.index)

for col in new_category_needed.index:
    all[col] = all[col].fillna("Other")

# Otherwise, impute with a SPECIFIC MODE that makes sense if possible, falling back to a general mode if not possible
def impute_mode(group):
    mode = group.mode()[0] # if multiple modes, choose 1st arbitrarily
    return group.fillna(mode)

for col in categorical_missing_cols:
    if col not in new_category_needed.index:
        try:
            all[col] = all.groupby(["Neighborhood", "MSSubClass"])[col].transform(impute_mode) # grouping by neighborhood and subclass makes sense
        except:
            all[col] = all[col].fillna(all[col].mode()[0])

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


In [67]:
# QUANTITATIVE IMPUTATION

# Again, special logic first (GarageYrBlt)
all["GarageYrBlt"] = all["GarageYrBlt"].fillna(all["YearBuilt"])

# Otherwise general logic is to impute via mean (can impute using something else, case by case)
    # Use a more specific mean when possible! Fall back on the general mean
for col, cat in quantitative_missing_cols_to_categories.items():
    try:
        all[col] = all[col].fillna(all.groupby(cat)[col].transform("mean"))
    except:
        all[col] = all[col].fillna(all[col].mean())

# Check that only Sale Price has missing data now
all.columns[all.isnull().any()]

Index(['SalePrice'], dtype='object')

**3. Feature engineering**

In [68]:
# Examine which raw quantitative features are well-correlated to SalePrice -> pandas corr, seaborn heatmap good
all.corr(numeric_only=True)["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
GarageYrBlt      0.508043
YearRemodAdd     0.507101
MasVnrArea       0.474904
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.329387
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePrice, dtype: float64

In [76]:
# One-hot encoding
all = pd.get_dummies(all)

In [77]:
# Create features that are interpretable -> i.e. everyone expects total square footage to correlate strongly with price
def add_total_sf(df):
    df["TotalSF"] = df["TotalBsmtSF"] + df["1stFlrSF"] + df["2ndFlrSF"]
    return df

def add_years_since_sold(df):
    df["YearsSinceSold"] = int(date.today().year) - df["YrSold"].astype(int)
    return df

def add_bathrooms(df):
    df["Bathrooms"] = 0.5 * df["HalfBath"] + df["FullBath"] + 0.5 * df["BsmtHalfBath"] + df["BsmtFullBath"]
    return df

all = add_total_sf(all)
all = add_years_since_sold(all)
all = add_bathrooms(all)

all[["SalePrice", "TotalSF", "YearsSinceSold", "Bathrooms"]].corr()

,SalePrice,TotalSF,YearsSinceSold,Bathrooms
SalePrice,1.000000,0.782260,0.028923,0.631731
TotalSF,0.782260,1.000000,0.022934,0.608636
YearsSinceSold,0.028923,0.022934,1.000000,-0.023450
Bathrooms,0.631731,0.608636,-0.023450,1.000000


**4. Choose & validate models**

In [ ]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

random_state = 123

def rmse(predicted, actual):
    return np.sqrt(np.mean((predicted - actual) ** 2))

X_train, X_valid, y_train, y_valid = train_test_split(all[:len(train)].drop("SalePrice", axis=1), all[:len(train)]["SalePrice"], test_size=0.2, random_state=random_state)

# Ensemble models
rf_search_space = {
    "max_depth": [3, 6, 9],
    "n_estimators": [100, 200, 300],
}

rf_model = RandomForestRegressor(random_state=random_state)
rf_cv = GridSearchCV(rf_model, rf_search_space)
rf_cv.fit(X_train, y_train)
print(f"Random Forest regressor RMSE: {rmse(rf_cv.predict(X_valid), y_valid)}")

gb_search_space = rf_search_space.copy()
gb_search_space["learning_rate"] = [0.001, 0.01, 0.1, 1]

gb_model = GradientBoostingRegressor(random_state=random_state)
gb_cv = GridSearchCV()


# Linear models


0.08904109589041095


In [81]:
all.iloc[:len(train), -1]


0       False
1       False
2       False
3       False
4       False
        ...  
1455    False
1456    False
1457    False
1458    False
1459    False
Name: SaleCondition_Partial, Length: 1460, dtype: bool